In [2]:
import DataFunctions
import numpy as np



Overall idea of simulation: Do pairings taking into consideration current score (win-lose rate) and their geographical location. It will use an adaptation of the Swiss Pairing System, the main difference is that instead of participants having a ELO score or rating and being paired by it; we consider an objective function on the total distance travelled and find proper pairings that minimize it. 

Tentative Objective function: $\sum (d_i)^2 + max(d_i)^2$. This function uses L2 norm and a L2 regularization term. We have to consider a regularization term as it helps us to avoid having multiple low distances and 1 large distance. For simplicity we can remove the regularization to trial the algorithm, and then regularization.

Rules per each round:
1. Split teams by their current score (#wins)
2. Pair from highest scoring group to lowest scoring group.
3. Per each group, find the group of pairs that minimize the objective function.
4. If there are odd # of teams, demote unpaired team into next scoring group. If it has been demoted before, use the next possible pairing that includes that team.
5. If there exist unpaired teams, demote them into the next scoring group. 
6. After making all possible pairings, set distance of teams that are playing together to inf.
7. For each pairing, if both teams have the same # of home games, pick one to be home and one to be away randomly. If one team has more # home games than the other, pick the team with the least # home games to be home, the other to be the away.

Things to store in db:
1. Each team 2022 stats.
2. Each team # (from 0 to 131).
3. Simulated season pairings, w-l record and h-a count.
4. Simulated games and scores.

Simulating each pairing:
1. Use 2022 stats to predict a spread and totalpts. We use those as means.
2. Using a normal distribution on the spread and totalpts with the standard deviation found on the model. We randomly generate a result with the simulated spread and totalpts.
3. We use obtained values to create an score.

In [4]:
#creating random distance matrix of size = 8
nteams = 8

m_dist = np.around(np.random.uniform(1,10,size=(nteams,nteams)),decimals=3)

for i in range(nteams):
    for j in range(nteams):
        m_dist[j,i]=m_dist[i,j]
    m_dist[i,i]=0

m_dist

array([[0.   , 2.98 , 4.498, 1.153, 1.375, 3.257, 8.012, 9.471],
       [2.98 , 0.   , 9.78 , 1.639, 5.744, 7.128, 1.194, 6.65 ],
       [4.498, 9.78 , 0.   , 2.93 , 4.544, 7.656, 8.781, 2.414],
       [1.153, 1.639, 2.93 , 0.   , 8.646, 7.559, 4.643, 6.742],
       [1.375, 5.744, 4.544, 8.646, 0.   , 2.224, 3.796, 9.647],
       [3.257, 7.128, 7.656, 7.559, 2.224, 0.   , 1.845, 8.895],
       [8.012, 1.194, 8.781, 4.643, 3.796, 1.845, 0.   , 9.279],
       [9.471, 6.65 , 2.414, 6.742, 9.647, 8.895, 9.279, 0.   ]])

In [5]:
#storing wins per team.
curr_data = np.zeros(shape=(nteams,2),dtype=int)
curr_data[:,0] = np.arange(nteams)
curr_data

array([[0, 0],
       [1, 0],
       [2, 0],
       [3, 0],
       [4, 0],
       [5, 0],
       [6, 0],
       [7, 0]])

In [6]:
#splitting in groups
sim1group0 = np.array([x[0].astype(int) for x in curr_data if x[1]==0])
# sim1group1 = [[x[0].astype(int),0] for x in curr_data if x[1]==1]
sim1group0

array([0, 1, 2, 3, 4, 5, 6, 7])

We cannot find all pairs and run for all of them. That operation has time complexity O(n!) and for 130ish teams is not feasible as it is about $10^{220}$. We need to use another approach.<br>
Fastest approach is done by sorting all possible distances, then picking the suitable one and remove unsuitables from the list. We only need to sort once since at the beginning all teams have to play. We should also count the # of possible matches and use that to do the sorting as well.<br><br>

Algorithm procedure using networkx and Blossom algorithm (also known as Edmonds' algorithm)

In [7]:
import networkx

In [110]:
L = []
for i in range(len(sim1group0)):
    for j in range(len(sim1group0)):
        if(i<j and m_dist[i,j]!=np.inf):
            L.append((-np.max(sim1group0[i,1],sim1group0[j,1]),m_dist[i,j],i,j))

LS = sorted(L)

In [111]:
LS

[(0, 1.147, 2, 3),
 (0, 1.522, 0, 4),
 (0, 1.61, 3, 4),
 (0, 1.717, 1, 7),
 (0, 1.919, 2, 6),
 (0, 2.189, 3, 7),
 (0, 2.391, 1, 4),
 (0, 2.66, 4, 6),
 (0, 2.976, 4, 5),
 (0, 3.054, 0, 5),
 (0, 3.2, 0, 6),
 (0, 3.628, 4, 7),
 (0, 4.053, 2, 5),
 (0, 4.262, 6, 7),
 (0, 4.297, 5, 7),
 (0, 5.376, 3, 6),
 (0, 5.735, 2, 7),
 (0, 6.252, 0, 1),
 (0, 6.407, 0, 7),
 (0, 7.043, 1, 6),
 (0, 7.168, 1, 3),
 (0, 7.632, 0, 2),
 (0, 8.552, 3, 5),
 (0, 8.641, 5, 6),
 (0, 9.171, 1, 5),
 (0, 9.217, 1, 2),
 (0, 9.479, 2, 4),
 (0, 9.554, 0, 3)]

In [103]:
def selection(LS,group):
    s = set(group[:,0])
    return s

In [102]:
selection(LS=LS,group=sim1group0)

{0, 1, 2, 3, 4, 5, 6, 7}